# Предобрабатываю данные (избавляюсь от дубликатов), агрегирую до года, соединяю таблицы

In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Collecting polars==1.22.0 (from -r requirements.txt (line 1))
  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/33.2 MB ? eta -:--:--
   -- ------------------------------------- 2.1/33.2 MB 11.7 MB/s eta 0:00:03
   ----- ---------------------------------- 4.5/33.2 MB 11.8 MB/s eta 0:00:03
   -------- ------------------------------- 7.1/33.2 MB 11.8 MB/s eta 0:00:03
   ----------- ---------------------------- 9.7/33.2 MB 11.8 MB/s eta 0:00:02
   -------------- ------------------------- 12.1/33.2 MB 11.8 MB/s eta 0:00:02
   ----------------- ---------------------- 14.7/33.2 MB 11.8 MB/s eta 0:00:02
   -------------------- ------------------- 17.0/33.2 MB 11.8 MB/s eta 0:00:02
   --------------------- ------------------ 17.8/3


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
cd ..

c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\data


c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import polars as pl

In [3]:
trade19 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2019_V202501.csv')
trade20 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2020_V202501.csv')
trade21 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2021_V202501.csv')
trade22 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2022_V202501.csv')
sql = pl.SQLContext(register_globals=True)

In [4]:
#Структура данных. t, i , j , k --- ключ.
sql.execute('''select * from trade19 limit 1;''').collect()

t,i,j,k,v,q
i64,i64,i64,i64,f64,f64
2019,4,20,420229,0.471,0.001


In [5]:
#Проверяю, что в данных нет дублей
print(sql.execute('''select i, j, k, count(*) as cnt from trade19 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade20 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade21 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade22 group by 1, 2, 3 order by 4 desc limit 1;''').collect())

shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 757 ┆ 528 ┆ 551511 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 757 ┆ 686 ┆ 580632 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 702 ┆ 196 ┆ 630491 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 604 ┆ 484 ┆ 620193 ┆ 1   │
└─────┴─────┴────────┴─────┘


In [6]:
sql.execute(''' 
            create table trade_tmp as
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade20
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade21
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade22
            ;
            ''').collect()
sql.execute('''select count(*) from trade_tmp;''').collect()

len
u32
34145929


In [7]:
del trade20
del trade21
del trade22

In [8]:
trade = sql.execute('''select 
                tr.*
                , t19.v as v19
                , t19.q as q19    
            from trade_tmp tr
            inner join trade19 t19
                on tr.i = t19.i
                and tr.j = t19.j
                and tr.k = t19.k
            ;''').collect()
sql.register('trade', trade)

<SQLContext [tables:6] at 0x1b055930590>

## Агрегирую заболеваемость и смертность

In [9]:
covid_tmp = pl.read_csv('raw_data/Our World In Data/owid-covid-data.csv')
sql.register('covid_tmp', covid_tmp)
sql.execute('''select * from covid_tmp limit 2;''').collect()

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
str,str,str,str,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,i64,str,str,str,str
"""AFG""","""Asia""","""Afghanistan""","""2020-01-05""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null
"""AFG""","""Asia""","""Afghanistan""","""2020-01-06""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null


In [10]:
#Надо подумать, нет ли тут эндогенных пропусков (т.е. если у страны нулевая заболеваемость, значит ли это, что она ничего не репортит, а не репортит 0)
sql.execute('''create table covid as
    select 
            iso_code as iso
            , left(date, 4)::smallint as t
            , avg(new_cases_per_million) as cases
            , avg(new_deaths_per_million) as deaths
            , avg(new_tests_per_thousand) as tests
            , avg(population_density) as population_density
            , avg(median_age) as median_age
            , avg(aged_65_older) as aged_65_older
            , avg(aged_70_older) as aged_70_older
            , avg(cardiovasc_death_rate) as cardiovasc_death_rate
            , avg(handwashing_facilities) as handwashing_facilities
            , avg(life_expectancy) as life_expectancy
    from covid_tmp
    where date <= '2022-12-01'
    group by iso_code, left(date, 4)        
;''')
sql.execute('''select * from covid limit 2;''').collect()

iso,t,cases,deaths,tests,population_density,median_age,aged_65_older,aged_70_older,cardiovasc_death_rate,handwashing_facilities,life_expectancy
str,i16,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""VNM""",2020,0.039834,0.000967,null,308.13,32.6,7.15,4.72,245.46,85.85,75.4
"""PCN""",2022,265.33997,0.0,null,null,null,null,null,null,null,null
